In [1]:
#### import packages ####
import os
import ipywidgets as widgets
from ipywidgets import Label, HBox, VBox
from ipyfilechooser import FileChooser

#### import MY packages ####
from parsers import RHSCETparser
from setups import FCpowermap
from setups import utilities
from setups import UIdisplay

In [2]:
### Create CASE ###
CASE = widgets.Text(value="TEST", description="CASE Name", disabled=False)
display(CASE)

Text(value='TEST', description='CASE Name')

In [15]:
### Create case in CASEs folder base on CASE Name ###
root = os.path.join(os.getcwd(), "RESULTs")
caseFolder = utilities.createCASE(root, CASE.value, subFolders=[])

## UT1: Thermal Profiles Display ##
1. Load DIE thermal profiles
2. Determine the highest temperature value
3. Display layer thermal profiles with grid-layout setting

In [44]:
import importlib
importlib.reload(UIdisplay)

TProfileList = []
out = UIdisplay.askFileDropItems2Processing("Select Files", TProfileList)
display(out)

In [51]:
import importlib
importlib.reload(RHSCETparser)

### Processing loaded thermal profiles ###
profileDict = {}
validProfiles = []
TMax, TMin = -100000, 100000
for p in TProfileList:
    profilePath = p[0]
    if os.path.isfile(profilePath):
        profileDict.setdefault(profilePath, {"OBJ":None,"PLOT":{"selected":[]}})
        validProfiles.append(profilePath)
        profileDict[profilePath]["OBJ"] = RHSCETparser.RHSCETView(profilePath, outputFolder=caseFolder)
    else:
        print("<ERROR> profile path not found")
        continue

    ### output DIE profile information ###
    TP = profileDict[profilePath]["OBJ"]
    print("DIE: {}, Design Area: {}".format(profilePath, [TP.llx, TP.lly, TP.urx, TP.ury]))
    #print("Available Layers: {}".format(TP.layerList))
    print("Max T within DIE: {}".format(TP.DIEMaxT))
    print("Min T within DIE: {}".format(TP.DIEMinT))

    if TMax < TP.DIEMaxT:
        TMax = TP.DIEMaxT
    if TMin > TP.DIEMinT:
        TMin = TP.DIEMinT

T_Upperbound = widgets.FloatText(value=TMax, description="T Upperbound:", \
                                 style={"description_width": "initial"}, disabled=False)
T_Lowerbound = widgets.FloatText(value=TMin, description="T Lowerbound:", \
                                 style={"description_width": "initial"}, disabled=False)
HBox([T_Upperbound, T_Lowerbound])

DIE: C:\Users\chienchi\OneDrive - Intel Corporation\Documents\PyCodes\DB\profile\Ly_exp\Arr_0509\9-9\ThermalProfile_DIE.txt, Design Area: [-45.0, -45.0, 45.0, 45.0]
Max T within DIE: 70.108
Min T within DIE: 55.923
DIE: C:\Users\chienchi\OneDrive - Intel Corporation\Documents\PyCodes\DB\profile\Ly_exp\Arr_0509\9-9\ThermalProfile_DIE_1.txt, Design Area: [-45.0, -45.0, 45.0, 45.0]
Max T within DIE: 70.344
Min T within DIE: 56.0


In [52]:
### Select profiles for displayment ###
_layout = []
for p in validProfiles:
    layerlist = profileDict[p]["OBJ"].layerList
    ui = UIdisplay.askDropItems2Processing("{}".format(p), layerlist, profileDict[p]["PLOT"]["selected"])
    _layout.append(ui)

isColorBar = widgets.Checkbox(value=False, description="with Colorbar?",\
                              style={"description_width": "initial"}, disabled=False)
_layout.append(isColorBar)
VBox(_layout)

In [53]:
### Plot the selected profiles ###
import importlib
importlib.reload(RHSCETparser)

imgList = []
for i, p in enumerate(validProfiles):
    profileObj = profileDict[p]["OBJ"]
    nickName = "DIE{}".format(str(i+1))
    for ly in profileDict[p]["PLOT"]["selected"]:
        imgPath = profileObj.plot(dieName=nickName, selectedLayer=ly,
                                  min_val=T_Lowerbound.value, max_val=T_Upperbound.value, withColorBar=isColorBar.value,
                                  saveImg="TProfile_{}_{}.png".format(nickName, ly), isShow=False)
        print(imgPath)
        imgList.append(imgPath)

C:\Users\chienchi\OneDrive - Intel Corporation\Documents\PyCodes\Thermal\RHSC_ET\RESULTs\TEST_org\TProfile_DIE1_NPTAB.png
C:\Users\chienchi\OneDrive - Intel Corporation\Documents\PyCodes\Thermal\RHSC_ET\RESULTs\TEST_org\TProfile_DIE2_NPTAB.png


In [54]:
gridX = widgets.IntText(value=1, description="Grid X:", style={"description_width": "initial"}, 
                        layout=widgets.Layout(width="100px"), disabled=False)
gridY = widgets.IntText(value=2, description="Grid Y:", style={"description_width": "initial"}, 
                        layout=widgets.Layout(width="100px"), disabled=False)
gridImgName = widgets.Text(value="merge.png", description="Save Name:", \
                           style={"description_width": "initial"}, disabled=False)
isShowImg = widgets.Checkbox(value=False, description="Show Image",\
                             style={"description_width": "initial"}, disabled=False)
HBox([gridX, gridY, gridImgName, isShowImg])

In [56]:
import importlib
importlib.reload(utilities)

utilities.saveGridImagesWithCBar(XX=gridX.value, YY=gridY.value, vmin=T_Lowerbound.value, vmax=T_Upperbound.value, \
                                 imgList=imgList, caseFolder=caseFolder, imgName=gridImgName.value, \
                                 isShowImg=isShowImg.value)


save image: C:\Users\chienchi\OneDrive - Intel Corporation\Documents\PyCodes\Thermal\RHSC_ET\RESULTs\TEST_org\merge.png


## UT2: Thermal/Power Correlation Analysis ##

In [4]:
DIEDict = {}

### select the thermal profiles ###
DIEDict["name"] = widgets.Text(value="DIE", description="DIE Name:", disabled=False)
DIEDict["resolution"] = widgets.IntText(value=1, description="CTM Resolution:", style={"description_width": "initial"}, disabled=False)
DIEDict["profilePath"] = FileChooser("./")
DIEDict["powermapPath"] = FileChooser("./")
_HB1 = HBox([DIEDict["name"], DIEDict["resolution"]])
_HB2 = HBox([Label("Thermal Profile :"), DIEDict["profilePath"]])
_HB3 = HBox([Label("ET_MHS Power File:"), DIEDict["powermapPath"]])
display(VBox([_HB1, _HB2, _HB3]))

In [16]:
## Parsing ET thermal profile ##
import importlib
importlib.reload(RHSCETparser)

profilePath = DIEDict["profilePath"].selected
if os.path.isfile(profilePath):
    DIEDict["RHSCET"] = RHSCETparser.RHSCETView(profilePath, outputFolder=caseFolder)
else:
    print("<ERROR> profile path not found")

### output DIE profile information ###
DIE = DIEDict["RHSCET"]
print("DIE: {}, Design Area: {}".format(DIEDict["name"].value, [DIE.llx, DIE.lly, DIE.urx, DIE.ury]))
print("Available Layers: {}".format(DIE.layerList))
print("Max T within DIE: {}".format(DIE.DIEMaxT))
print("Min T within DIE: {}".format(DIE.DIEMinT))

DIE: DIE, Design Area: [-45.0, -45.0, 45.0, 45.0]
Available Layers: ['BM4', 'BV3', 'BM3', 'BV2', 'BM2', 'BV1', 'BM1', 'BV0', 'BM0', 'TSV', 'POD', 'NOD', 'NPTAB', 'M0', 'VIA0', 'M1', 'VIA1', 'M2', 'VIA2', 'M3', 'VIA3', 'M4', 'VIA4', 'M5', 'VIA5', 'M6', 'VIA6', 'M7', 'VIA7', 'M8', 'VIA8', 'M9', 'VIA9', 'M10', 'VIA10', 'M11']
Max T within DIE: 188.418
Min T within DIE: 171.726


In [17]:
## Translate input power map to tile-based power map ##
import importlib
importlib.reload(FCpowermap)

pwrPath = DIEDict["powermapPath"].selected
if os.path.isfile(pwrPath):
    resolution = DIEDict["resolution"].value
    DIEDict["ETMHS"] = FCpowermap.FCPowerView(pwrPath, "ETMHS", outputFolder=caseFolder)
    DIEDict["ETMHS"].translate2TilePwr(resolution=resolution)
else:
    print("<ERROR> power map path not found")

TPimgName = widgets.Text(value="TilePwr_{}um.png".format(str(resolution)), description="Tile Power Image Name:", \
                         style={"description_width": "initial"}, disabled=False)
isShowTilePwrImg = widgets.Checkbox(value=False, description="Show Tile-based power map",\
                                    style={"description_width": "initial"}, disabled=False)
HBox([TPimgName, isShowTilePwrImg])
#DIEDict["ETMHS"].plot(ptype=["TILE", resolution], saveImg="TilePwr_{}um.png".format(str(resolution)))

<INFO> Instances Area:-45.0,-45.0,45.0,45.0; Total PWR=12.000mW


In [18]:
## generate tile-based power image ##
TPimgPath = DIEDict["ETMHS"].plot(ptype=["TILE", resolution], saveImg=TPimgName.value)

if isShowTilePwrImg.value:
    utilities.viewImg(TPimgPath)

In [13]:
### Select the layer of thermal profile to process the negative Laplacian transform ###
layer_dropdown = widgets.Dropdown(options=DIEDict["RHSCET"].layerList, description="Layers:", 
                                  style={"description_width": "initial"})
display(layer_dropdown)

Dropdown(description='Layers:', options=('BM4', 'BV3', 'BM3', 'BV2', 'BM2', 'BV1', 'BM1', 'BV0', 'BM0', 'TSV',…

In [19]:
from setups import thermalAnalysis

import importlib
importlib.reload(thermalAnalysis)

ly = layer_dropdown.value
thermalCorr = thermalAnalysis.powerThermalAnalysis(DIEDict["ETMHS"], DIEDict["RHSCET"], \
                                                   resolution=resolution, outputFolder=caseFolder)
thermalCorr.calNegLaplacianOperator(layer=ly)
thermalCorr.gradPlot(layer=ly, saveImg="grad_{}_{}_{}um.png".format(DIEDict["name"].value, ly, str(DIEDict["resolution"].value)))
thermalCorr.outputCorrDict(layer=ly)

#layer = "NPTAB"
#thermalCorr = thermalAnalysis.powerThermalAnalysis(DIEDict["ETMHS"], DIEDict["RHSCET"], resolution=DIEDict["resolution"].value)
#thermalCorr.calNegLaplacianOperator(layer=layer)
#thermalCorr.plot(saveImg="grad{}_{}um.png".format(layer, str(DIEDict["resolution"].value)))

#thermalCorr.calProfileCorr(layer="NPTAB", unit=5)
#thermalCorr.plot()

Min Grad: -1.389, Max Grad: 10.846, Sum: 0.0, R: 0.74


In [25]:
import importlib
importlib.reload(RHSCETparser)
importlib.reload(UIdisplay)

for i in DIEDict.keys():
    DIEDict[i]["layerList"] = []
    title = DIEDict[i]["name"].value
    out = UIdisplay.askDropItems2Processing(title=title, layerList=DIEDict[i]["RHSCET"].layerList, \
                                            layerQ=DIEDict[i]["layerList"])
    display(out)

#layerQ = []
#out = UIdisplay.askLayers2Processing(TD1.layerList, layerQ)
#display(out)

#file = open("./MEOL.png", "rb")
#img = file.read()
#widgets.Image(value=img, format="png", width=400, height=400)

In [29]:
T_Max = T_Upperbound.value

for i in DIEDict.keys():
    name = DIEDict[i]["name"].value
    DIE = DIEDict[i]["RHSCET"]
    layers = list(set(DIEDict[i]["layerList"]))
    for ly in layers:
        #hots = TD1.getTopNHotSpots(selectedLayer=ly)
        hots = []
        saveImgName = "{}_{}.png".format(name, ly)
        DIE.plot(dieName=name, selectedLayer=ly, max_val=T_Max, highlighted=hots, saveImg=saveImgName)


#layerQ = list(set(layerQ))
#print(layerQ)